#**Step 1: Installazione, caricamento delle librerie e definizione delle variabili**

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install accelerate -U

#!pip install evaluate
#!pip install bert_score
#!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
#from evaluate import load
#from torchmetrics.text import TranslationEditRate

In [3]:
percentuale_dataset = 1
dataset_type = ""
model_name = "t5-large"

#**Step 2: Collegamento a drive**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

# change this to match your path
data_path_models = "./drive/MyDrive/Colab Notebooks/WebNLG-IT/Models"
data_path_data = "./drive/MyDrive/Colab Notebooks/data"

Mounted at /content/drive


#**Step 3: Importazione dei dataset**

In [5]:
class CustomDataset(Dataset):
    def __init__(self, X, y, tokenizer):
        self.X = X
        self.y = y
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        input_text = self.X[idx]
        target_text = self.y[idx]

        input_encoding = self.tokenizer(input_text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
        target_encoding = self.tokenizer(target_text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')

        return {
            'input_ids': input_encoding['input_ids'].squeeze(),
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'labels': target_encoding['input_ids'].squeeze()
        }

# carico il dataset
trainset = pd.read_csv(f'{data_path_data}/WebNLG-RU/train.csv')
devset = pd.read_csv(f'{data_path_data}/WebNLG-RU/dev.csv')

In [6]:
# ottengo un sottoinsieme del dataset di partenza
subset_trainset = trainset.sample(frac=percentuale_dataset, random_state=42)
subset_devset = devset.sample(frac=percentuale_dataset, random_state=42)

# train
X_train = subset_trainset['data_unit'].values.tolist()
y_train = subset_trainset['sentence'].values.tolist()

# dev
X_val = subset_devset['data_unit'].values.tolist()
y_val = subset_devset['sentence'].values.tolist()

In [7]:
subset_trainset.head()

,data_unit,sentence
22823,National_League_North champions Barrow_A.F.C.,"""Барроу А.Ф.К."" одержал победу на чемпионате п..."
31240,Abdulsalami_Abubakar birthPlace Niger_State,Местом рождения Абдулсалами Абубакара было гос...
33615,Asser_Levy_Public_Baths ReferenceNumber in the...,Asser Levy Public Baths находится на Asser Lev...
19786,"Alfred_Moore_Scales office (workedAt, workedAs...",Губернатор Северной Каролины Альфред Мур Скале...
1758,"Alan_Bean birthDate ""1932-03-15"" Alan_Bean bir...","Alan Bean родился в Wheeler, Texas on Mar 15, ..."


In [8]:
len(X_train)

36234

#**Step 4: Caricamento del modello**

In [9]:
# Load the T5 model, tokenizer, and configuration
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)
config = T5Config.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
# Create the custom datasets
train_dataset = CustomDataset(X_train, y_train, tokenizer)
val_dataset = CustomDataset(X_val, y_val, tokenizer)

In [11]:
# Set up the data collator, training arguments, and trainer
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=tokenizer.pad_token_id
)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    #per_device_train_batch_size=2,
    #per_device_eval_batch_size=2,
    num_train_epochs=2,
    logging_dir="./logs",
    evaluation_strategy="steps",  # Calcolo della validation loss ad ogni epoca
    save_strategy="steps",  # Checkpoint del modello ad ogni epoca
    save_total_limit=3,  # Limita il numero di checkpoint salvati
    load_best_model_at_end=True,  # Carica il modello migliore al termine dell'addestramento
    metric_for_best_model="eval_loss",  # Sceglie la metrica per determinare il miglior modello
    greater_is_better=False,  # Indica se un valore più alto della metrica è migliore o no
    logging_steps=500,
    eval_steps=500,  # Numero di passaggi prima di valutare il modello
    # Aggiungi eventuali altri argomenti di training qui
)

In [12]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

Step,Training Loss,Validation Loss
500,1.342700,0.824444
1000,0.875300,0.667082
1500,0.731100,0.571495
2000,0.648300,0.509496
2500,0.599200,0.466499
3000,0.547300,0.434840
3500,0.518800,0.409339
4000,0.497300,0.391673
4500,0.471800,0.375830
5000,0.445500,0.365016


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=9060, training_loss=0.551444737432282, metrics={'train_runtime': 7010.0755, 'train_samples_per_second': 10.338, 'train_steps_per_second': 1.292, 'total_flos': 3.9224161861632e+16, 'train_loss': 0.551444737432282, 'epoch': 2.0})

In [13]:
# salvataggio del modello
model.save_pretrained(f'{data_path_models}/{model_name}_ru_2epoch')
tokenizer.save_pretrained(f'{data_path_models}/{model_name}_ru_2epoch')

('./drive/MyDrive/Colab Notebooks/WebNLG-IT/Models/t5-large_ru_2epoch/tokenizer_config.json',
 './drive/MyDrive/Colab Notebooks/WebNLG-IT/Models/t5-large_ru_2epoch/special_tokens_map.json',
 './drive/MyDrive/Colab Notebooks/WebNLG-IT/Models/t5-large_ru_2epoch/spiece.model',
 './drive/MyDrive/Colab Notebooks/WebNLG-IT/Models/t5-large_ru_2epoch/added_tokens.json')

#**Step 5: generazione frasi del modello creato**

In [14]:
model1 = T5ForConditionalGeneration.from_pretrained(f'{data_path_models}/{model_name}_ru_2epoch')
tokenizer1 = T5Tokenizer.from_pretrained(f'{data_path_models}/{model_name}_ru_2epoch')
print(f'{data_path_models}/{model_name}_ru_2epoch')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


./drive/MyDrive/Colab Notebooks/WebNLG-IT/Models/t5-large_ru_2epoch


In [15]:
def generate_text(input_rdf, max_length=128):
  input_text = f"{input_rdf}"
  input_ids = tokenizer1.encode(input_text, return_tensors="pt")

  # Imposta il parametro max_length per controllare la lunghezza dell'output generato
  output_ids = model1.generate(input_ids, max_length=max_length)

  output_text = tokenizer1.decode(output_ids[0], skip_special_tokens=True)
  return output_text

In [16]:
testset = pd.read_csv(f'{data_path_data}/WebNLG-RU/test.csv')

# ottengo un sottoinsieme del testset di partenza
subset_testset = testset.sample(frac=percentuale_dataset, random_state=42)

# test
X_test = subset_testset['data_unit'].values.tolist()
y_test = subset_testset['sentence'].values.tolist()

len(X_test)

1000

In [17]:
subset_testset.head()

,data_unit,sentence
521,Peter_Stöger club FC_Admira_Wacker_Mödling,"Клуб ""Питер Стёгер"" - это ФК ""Адмира Wacker Mö..."
737,Arem-arem ingredient Banana_leaf,Банановый лист - это ингредиент в Arem arem.
740,Weymouth_Sands followedBy Maiden_Castle_(novel...,"Следом за ""Гластонбери Романс"" последовали Wey..."
660,Christian_Panucci club Genoa_C.F.C. Christian_...,Christian Panucci играет за Genoa C.C. и сборн...
411,Chicharrón region Andalusia,Chicharrón найден в районе Андалусия.


In [18]:
triple = []
predicted = []
actuals = []

i = 0
for tripla in X_test:
  triple.append(tripla)
  prediction = generate_text(tripla)
  predicted.append(prediction)
  actuals.append(y_test[i])

  print(f'{i}/{len(X_test)-1}')
  print('tripla: ', tripla)
  print('actual: ', y_test[i])
  print('predicted: ', prediction)
  print('\n')

  df = pd.DataFrame(list(zip(triple, predicted, actuals)), columns =['triple', 'predicted', 'actuals'])
  df.to_csv(f'{data_path_models}/{model_name}_ru_2epoch_decoding.csv', index=False)

  i += 1



Output streaming troncato alle ultime 5000 righe.


167/999
tripla:  Andrews_County_Airport runwayLength 1773.0
actual:  Длина взлетно-посадочной полосы в аэропорту Эндрюс - 1773,0.
predicted:  Andrews County Airport имеет длину влетно-осадоно олос 1773,0.


168/999
tripla:  Alderney_Airport runwayLength 497.0
actual:  Alderney Airport имеет длину взлетно-посадочной полосы 497,0.
predicted:  ротенност влетно-осадоно олос ароорта "лдерни" составлет 497,0.


169/999
tripla:  A.C._Chievo_Verona ground "Verona, Italy" A.C._Chievo_Verona league Serie_A
actual:  Верона, Италия - родина A.C. Chievo Verona, сыгранная в Серии А.
predicted:  A.C. Chievo Verona ирает в ерии  в ероне, тали.


170/999
tripla:  Auron_(comicsCharacter) creator Karl_Kesel Auron_(comicsCharacter) creator Walt_Simonson
actual:  Комический персонаж "Аурон" был создан как Карлом Кезелем, так и Уолтом Симонсоном.
predicted:  ерсона комиксов "урон" л содан арл еел и арли имонсон.


171/999
tripla:  Adolfo_Suárez_Madrid–Bara